In [2]:
import os
from sor.machine_learning import *

In [3]:
sor_data=pd.read_csv('C:\\Users\\surgi\\source\\repos\\Speech_Omics_data.txt',sep='\t',index_col=0)
allf=sor_data.columns.to_list()

In [4]:
sor_cca=pd.read_csv('C:\\Users\\surgi\\source\\repos\\Speech_Omics_Modularity.txt',sep='\t',index_col=0)

In [5]:
task=pd.ExcelFile('C:\\Users\\surgi\\source\\repos\\0.basic\\Task27.xlsx')

In [6]:
fc=['ALL', 'FCR', 'VSA', 'F2R', 'F1a', 'F2a', 'F3a', 'B1a', 'B2a', 'B3a', 'F1i', 'F2i', 'F3i', 'B1i', 'B2i', 'B3i', 'F1u', 'F2u', 'F3u', 'B1u', 'B2u', 'B3u']

In [7]:
feat_dict={}
for fn in range(5):
    dicf={}
    with open('C:\\Users\\surgi\\source\\repos\\0.basic\\features_r{}.txt'.format(str(fn+1)),'r') as f:
        for ff in f.readlines():
            dicf[ff.strip('\n').split(':')[0]]=ff.strip('\n').split(':')[-1].split('+')
    feat_dict['r'+str(fn+1)]=dicf

In [ ]:
for t in tqdm(task.sheet_names):
    cdata=task.parse(sheet_name=t,index_col=0)
    sd0=sor_data.loc[cdata.index.to_list()]
    sc0=pd.concat([sor_cca,cdata],axis=1,join='inner')
    writer1 = pd.ExcelWriter('F:\\backup_thesis\\data\\pca_model\\ML\\{}_outcomes.xlsx'.format(t))
    for fn,fs in feat_dict.items():
        sc_sd=pd.concat([sd0,sc0],axis=1)
        sr0=sc_sd[fs[t]]
        sr00=pd.concat([sr0,cdata],axis=1)
        sr_train=sr00.loc[sr00['Training set']=='Yes']
        sr_test=sr00.loc[sr00['Training set']=='No']
        pmodel_n, pdval, aval=MLSS(data=sr_train,data1=sr_test,outpath='F:\\backup_thesis\\data\\pca_model\\ML',cols=fs[t],var=[t],model_name=t,prefix=fn)
        pdval['model'] = [str.split(i, '_')[0] for i in pdval.index.to_list()]
        pdval['state'] = ['_'.join(str.split(i, '_')[1:]) for i in pdval.index.to_list()]
        pdval.sort_values(by=['state', 'Test_accuracy'], ascending=False, inplace=True)
        pdval.to_excel(writer1, 'test_{}'.format(fn))
        pmodel_n['mean_value'] = pmodel_n.mean(axis=1)
        pmodel_n['std'] = pmodel_n.std(axis=1)
        pmodel_n['cv'] = pmodel_n['std'] / pmodel_n['mean_value']
        pmodel_n['parameter'] = ['_'.join(str.split(i, '_')[2:]) for i in pmodel_n.index.to_list()]
        pmodel_n['model'] = [str.split(i, '_')[0] for i in pmodel_n.index.to_list()]
        pmodel_n['state'] = ['_'.join(str.split(i, '_')[1:-1]) for i in pmodel_n.index.to_list()]
        pmodel_n.sort_values(by=['state', 'parameter', 'mean_value'], ascending=False, inplace=True)
        pmodel_n.to_excel(writer1, 'training_{}'.format(fn))
        aval.to_excel(writer1, 'individual_pred&prob_{}'.format(fn))
    for fn,fs in zip(['CCA']+fc,[sor_cca.columns.to_list()]+[[fd for fd in sor_cca.columns.to_list() if fd0 in fd.split('-')] for fd0 in fc]):
        sc_train=sc0.loc[sc0['Training set']=='Yes']
        sc_test=sc0.loc[sc0['Training set']=='No']
        pmodel_n, pdval, aval=MLSS(data=sc_train,data1=sc_test,outpath='F:\\backup_thesis\\data\\pca_model\\ML',cols=fs,var=[t],model_name=t,prefix=fn)
        pdval['model'] = [str.split(i, '_')[0] for i in pdval.index.to_list()]
        pdval['state'] = ['_'.join(str.split(i, '_')[1:]) for i in pdval.index.to_list()]
        pdval.sort_values(by=['state', 'Test_accuracy'], ascending=False, inplace=True)
        pdval.to_excel(writer1, 'test_{}'.format(fn))
        pmodel_n['mean_value'] = pmodel_n.mean(axis=1)
        pmodel_n['std'] = pmodel_n.std(axis=1)
        pmodel_n['cv'] = pmodel_n['std'] / pmodel_n['mean_value']
        pmodel_n['parameter'] = ['_'.join(str.split(i, '_')[2:]) for i in pmodel_n.index.to_list()]
        pmodel_n['model'] = [str.split(i, '_')[0] for i in pmodel_n.index.to_list()]
        pmodel_n['state'] = ['_'.join(str.split(i, '_')[1:-1]) for i in pmodel_n.index.to_list()]
        pmodel_n.sort_values(by=['state', 'parameter', 'mean_value'], ascending=False, inplace=True)
        pmodel_n.to_excel(writer1, 'training_{}'.format(fn))
        aval.to_excel(writer1, 'individual_pred&prob_{}'.format(fn))
    for th in range(2, 10):
        threshold = th / 10
        for interval in range(10, 101, 10):
            sd00=sd0[[col for col in allf if col.split('_')[-2]==str(threshold) and col.split('_')[-1]==str(interval)]]
            sds=pd.concat([sd00,cdata],axis=1)
            sd_train=sds.loc[sds['Training set']=='Yes']
            sd_test=sds.loc[sds['Training set']=='No']
            use_fs1=[f for f in allf if (f.split('_')[1]=='ALL' and f.split('_')[-2]==str(threshold) and f.split('_')[-1]==str(interval))]
            use_fs2=[f for f in allf if (f.split('_')[1]!='ALL' and int(f.split('_')[2])<=3 and f.split('_')[-2]==str(threshold) and f.split('_')[-1]==str(interval))]
            use_fs3=[f for f in allf if (int(f.split('_')[2])<=3 and f.split('_')[-2]==str(threshold) and f.split('_')[-1]==str(interval))]
            use_fs4=[f for f in allf if (f.split('_')[-2]==str(threshold) and f.split('_')[-1]==str(interval))]
            for fn,fs in zip(['Data'+str(xx) for xx in range(1,5)],[use_fs1,use_fs2,use_fs3,use_fs4]):
                pmodel_n, pdval, aval=MLSS(data=sd_train,data1=sd_test,outpath='F:\\backup_thesis\\data\\pca_model\\ML',cols=fs,var=[t],model_name=t,prefix=fn+'_'+str(threshold)+'_'+str(interval))
                pdval['model'] = [str.split(i, '_')[0] for i in pdval.index.to_list()]
                pdval['state'] = ['_'.join(str.split(i, '_')[1:]) for i in pdval.index.to_list()]
                pdval.sort_values(by=['state', 'Test_accuracy'], ascending=False, inplace=True)
                pdval.to_excel(writer1, 'test_{}_{}_{}'.format(str(threshold),str(interval),fn))
                pmodel_n['mean_value'] = pmodel_n.mean(axis=1)
                pmodel_n['std'] = pmodel_n.std(axis=1)
                pmodel_n['cv'] = pmodel_n['std'] / pmodel_n['mean_value']
                pmodel_n['parameter'] = ['_'.join(str.split(i, '_')[2:]) for i in pmodel_n.index.to_list()]
                pmodel_n['model'] = [str.split(i, '_')[0] for i in pmodel_n.index.to_list()]
                pmodel_n['state'] = ['_'.join(str.split(i, '_')[1:-1]) for i in pmodel_n.index.to_list()]
                pmodel_n.sort_values(by=['state', 'parameter', 'mean_value'], ascending=False, inplace=True)
                pmodel_n.to_excel(writer1, 'training_{}_{}_{}'.format(str(threshold),str(interval),fn))
                aval.to_excel(writer1, 'individual_pred&prob_{}_{}_{}'.format(str(threshold),str(interval),fn))
    writer1._save()
    time.sleep
    

  4%|▎         | 1/27 [25:02:02<650:52:59, 90122.27s/it]